In [2]:
# Importing the necessary libraries
import pandas as pd
import numpy as np

In [3]:
train_dir=r'D:\amazonml\student_resource 3\dataset\train.csv'
df=pd.read_csv(train_dir)

In [4]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [15]:
df['image_link'][0]

'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'

In [5]:
df['entity_name'].value_counts()

entity_name
item_weight                      102786
depth                             45127
width                             44183
height                            43597
voltage                            9466
wattage                            7755
item_volume                        7682
maximum_weight_recommendation      3263
Name: count, dtype: int64

In [21]:
import os
from pathlib import Path
import urllib
import time

def create_placeholder_image(image_save_path):
    from PIL import Image  
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        print(f"Error creating placeholder image: {e}")
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        print("Invalid image link:", image_link)
        return

    # Ensure the directory exists
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
        print(f"Directory {save_folder} created.")

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        print(f"Image already exists: {image_save_path}")
        return

    for attempt in range(retries):
        try:
            print(f"Attempting to download image: {image_link}, attempt {attempt + 1}")
            urllib.request.urlretrieve(image_link, image_save_path)
            print(f"Image successfully downloaded: {image_save_path}")
            return
        except Exception as e:
            print(f"Download failed on attempt {attempt + 1}: {e}")
            time.sleep(delay)
    
    print(f"Failed to download image after {retries} attempts. Creating placeholder image.")
    create_placeholder_image(image_save_path)


In [38]:
for i in range(1,10):
    download_image(image_link=df['image_link'][i],save_folder='images')

Attempting to download image: https://m.media-amazon.com/images/I/71gSRbyXmoL.jpg, attempt 1
Image successfully downloaded: images\71gSRbyXmoL.jpg
Attempting to download image: https://m.media-amazon.com/images/I/61BZ4zrjZXL.jpg, attempt 1
Image successfully downloaded: images\61BZ4zrjZXL.jpg
Attempting to download image: https://m.media-amazon.com/images/I/612mrlqiI4L.jpg, attempt 1
Image successfully downloaded: images\612mrlqiI4L.jpg
Attempting to download image: https://m.media-amazon.com/images/I/617Tl40LOXL.jpg, attempt 1
Image successfully downloaded: images\617Tl40LOXL.jpg
Attempting to download image: https://m.media-amazon.com/images/I/61QsBSE7jgL.jpg, attempt 1
Image successfully downloaded: images\61QsBSE7jgL.jpg
Attempting to download image: https://m.media-amazon.com/images/I/81xsq6vf2qL.jpg, attempt 1
Image successfully downloaded: images\81xsq6vf2qL.jpg
Attempting to download image: https://m.media-amazon.com/images/I/71DiLRHeZdL.jpg, attempt 1
Image successfully downlo

In [17]:
df['image_link'][0]

'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'

In [62]:
def read_text_from_image(local_image_path):
    if not isinstance(local_image_path,str):
        print("Please give a valid image path")
        # returning nan if the path is invalic
        return np.nan
    else:
        import pytesseract
        try:

            from PIL import Image
            img=Image.open(local_image_path)
            text = pytesseract.image_to_string(img).strip()
            if not text:  # Check if the text is empty after stripping
                print(f"No text could be extracted from {local_image_path}")
                return np.nan
            else:
                print(f"Text extracted from {local_image_path}:\n{text}\n")
                return text
            
        except Exception as e:
            print(f"Error processing {local_image_path}: {e}")
            return np.nan 

In [63]:
def return_all_file_names(directory_path):
    """
    Return all corresponding Image paths of a directory
    """
    if not isinstance(directory_path,str):
        print("Not a valid path")
        return 
    try:
        for file_name in os.listdir(directory_path):
            full_path=os.path.join(directory_path,file_name)
            if os.path.isfile(full_path):
                yield os.path.join('images',file_name)
                
    except FileNotFoundError:
        print(f"Directory not found: {directory_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


In [64]:
def create_image_text_dataframe(directory_path):
    data={"Image Path":[],"Extracted Text":[]}
    for file_name in return_all_file_names(directory_path):
        text = read_text_from_image(file_name)  
        data["Image Path"].append(file_name)    
        data["Extracted Text"].append(text)
    df = pd.DataFrame(data)
    return df

In [65]:
directory_path = r'D:\amazonml\student_resource 3\approach\images'

# Create the DataFrame
df = create_image_text_dataframe(directory_path)

# Display the DataFrame
print(df)

No text could be extracted from images\612mrlqiI4L.jpg
Text extracted from images\617Tl40LOXL.jpg:
HIGH STRENGTH

PSYLLIUM

HUSK

Text extracted from images\61BZ4zrjZXL.jpg:
Serving Size: 1 Tablet (0.709 g) | Each serving contains (Approx. Values):

Ingredient Oty. / Serving

*PHOSPHOcomplex® Silybin (Sillybum marianum) 200 mg
Dandelion (Taraxacum officinale) leaf extract - 10:1 100 mg
Kutki (Picrorhiza kurroa)rhizome extract - 0.5% Bitters 50 mg
Kasani (Cichorium intybus) seed extract - 1% Bitters 25 mg
Punarnava (Boerhavia diffusa) root extract - 0.07% alkaloids 25 mg
Bhui amla (Phyllanthus amarus) WP extract - 0.5% Bitters 25 mg
Amla (Phyllanthus emblica) fruit extract - 10% Tannins 25 mg
Licorice (Glycyrrhiza glabra) root extract - 5% Glycyrrhizin 25 mg
Vitamin E 10 mg
Piper nigrum fruit extract — 95% Piperine 5mg

Nutrients Qty. / Serving

Energy 3.04 kcal
Carbohydrate 051g
(Sugars) 02g
Protein 0.049
Fat 0.09 g

"ZRDA values established as per ICMR 2010 for sedentary lifestyle-Men

In [66]:
df

,Image Path,Extracted Text
0,images\612mrlqiI4L.jpg,NaN
1,images\617Tl40LOXL.jpg,HIGH STRENGTH\n\nPSYLLIUM\n\nHUSK
2,images\61BZ4zrjZXL.jpg,Serving Size: 1 Tablet (0.709 g) | Each servin...
3,images\61I9XdN6OFL.jpg,100% NATUR
4,images\61QsBSE7jgL.jpg,¢ Naturally-Sourced Psyllium\n\n—_\n\n* High s...
5,images\71DiLRHeZdL.jpg,©)\n\nVEGAN\n\nWO\n\nWHEAT\nFREE\n\nSOY\n\npsy...
6,images\71gSRbyXmoL.jpg,GEPRAGTES\n\nDesigned in\n\nLIZENZIERTE UND GE...
7,images\71jBLhmTNlL.jpg,NEW LOOK.\nSAME Nea LES QUALITY.\n\nMame PSYLL...
8,images\81xsq6vf2qL.jpg,"Directions: For adults, take two (2) vegan cap..."
9,images\91Cma3RzseL.jpg,Horbaach\n\n100%\n\nHIGHEST\nQUALITY\n\nHIGH S...


In [76]:
def extract_numbers_and_next_words(text):
    """
    args:
    :text -> It should be the extracted text column in dataframe, 
    Here I am trying to keep only the numbers present and next words (Hope, it's accurate)
    """
    if not isinstance(text,str):
        """
        Check to make sure that
        if I have some other kind of thing(than text)
        Example: np.Nan which is considered to be float
        """
        return np.nan
    try:
        import re
    except ImportError as e:
        print("re could not be imported")
        return 
    match = re.search(r'(\d+)\s*(\w+)?\s*(\w+)?', text)
    if match:
        # Construct the result string
        number = match.group(1)
        word1 = match.group(2) if match.group(2) else ""
        word2 = match.group(3) if match.group(3) else ""
        return f"{number} {word1} {word2}".strip()
    return np.nan

In [77]:
df['Extracted Text']=df['Extracted Text'].apply(extract_numbers_and_next_words)

In [78]:
df

,Image Path,Extracted Text
0,images\612mrlqiI4L.jpg,NaN
1,images\617Tl40LOXL.jpg,NaN
2,images\61BZ4zrjZXL.jpg,1 Tablet
3,images\61I9XdN6OFL.jpg,100
4,images\61QsBSE7jgL.jpg,1400 mg per
5,images\71DiLRHeZdL.jpg,365
6,images\71gSRbyXmoL.jpg,NaN
7,images\71jBLhmTNlL.jpg,4 FOOD SUPPLEMENT
8,images\81xsq6vf2qL.jpg,2
9,images\91Cma3RzseL.jpg,100


In [67]:
try:
    df.to_csv('extracted_text.csv',index=False)
except Exception as e:
    print("Sorry an exception occured")

In [79]:
import re
import numpy as np
import pandas as pd

def extract_all_numbers_and_words(text):
    if not isinstance(text, str):
        return np.nan
    
    # Regex to match any number followed by one or more words
    pattern = r'(\d+[\.\d+]*)\s*(\w+)(?:\s+(\w+))?'
    matches = re.findall(pattern, text)
    
    if matches:
        # Flatten the matches into a string format "number unit"
        result = []
        for match in matches:
            result.append(' '.join([part for part in match if part]))  # Join number and following words
        return ' '.join(result)  # Return all matches as a single string
    return np.nan

# Example text
text = '''
Serving Size: 1 Tablet (0.709 g) | Each serving contains (Approx. Values):

Ingredient Oty. / Serving

*PHOSPHOcomplex® Silybin (Sillybum marianum) 200 mg
Dandelion (Taraxacum officinale) leaf extract - 10:1 100 mg
Kutki (Picrorhiza kurroa)rhizome extract - 0.5% Bitters 50 mg
Kasani (Cichorium intybus) seed extract - 1% Bitters 25 mg
Punarnava (Boerhavia diffusa) root extract - 0.07% alkaloids 25 mg
Bhui amla (Phyllanthus amarus) WP extract - 0.5% Bitters 25 mg
Amla (Phyllanthus emblica) fruit extract - 10% Tannins 25 mg
Licorice (Glycyrrhiza glabra) root extract - 5% Glycyrrhizin 25 mg
Vitamin E 10 mg
Piper nigrum fruit extract — 95% Piperine 5mg

Nutrients Qty. / Serving

Energy 3.04 kcal
Carbohydrate 051g
(Sugars) 02g
Protein 0.049
Fat 0.09 g
'''
# Apply the function to the example
cleaned_text = extract_all_numbers_and_words(text)
print(cleaned_text)


1 Tablet 0.709 g 200 mg Dandelion 1 0 1 100 mg 0. 5 50 mg Kasani 25 mg Punarnava 0.0 7 25 mg Bhui 0. 5 25 mg Amla 1 0 25 mg Licorice 25 mg Vitamin 10 mg Piper 9 5 5 mg Nutrients 3.04 kcal Carbohydrate 051 g 02 g Protein 0.049 Fat 0 09 g
